# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
df.printSchema()
df.show(5)


Setting default log level to "

root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: string (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: string (nullable = true)
 |-- EXPIRED: string (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: string (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [2]:
# Your Code for 1st question here
pdf = df.select("EMPLOYMENT_TYPE_NAME", "SALARY_FROM")\
        .filter((df.SALARY_FROM.isNotNull()) & (df.SALARY_FROM != 0))\
        .toPandas()
fig = px.box(pdf, x="EMPLOYMENT_TYPE_NAME", y="SALARY_FROM",
             title="Salary Distribution by Employment Type",
             color="EMPLOYMENT_TYPE_NAME",
             notched=True,
             color_discrete_sequence=px.colors.qualitative.Prism,
             labels={
                 "EMPLOYMENT_TYPE_NAME": "employment type",
                 "SALARY_FROM": "salary",
             })
fig.update_layout(font_family="Times New Roman",
                  title_font_size=20,
                  width=1000,
                  height=600)
fig.write_image("_output/figure-q1.svg")

![Salary Distribution](_output/figure-q1.svg)  


Insights from this visualization:

1. full-time jobs are paid with the highest salary on average
2. the highest salary is only available to full-time workers

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [3]:
# Your code for 2nd question here
pdf = df.select("NAICS2_NAME", "SALARY_FROM")\
        .filter(df.SALARY_FROM > 0)\
        .toPandas()
fig = px.box(pdf, x="NAICS2_NAME", y="SALARY_FROM",
             title="Salary Distribution by Industry",
             color="NAICS2_NAME",
             color_discrete_sequence=px.colors.qualitative.Prism,
             labels={
                 "NAICS2_NAME": "industry",
                 "SALARY_FROM": "salary",
             })
fig.update_layout(font_family="Times New Roman",
                  title_font_size=20,
                  width=1000,
                  height=1200)
fig.write_image("_output/figure-q2.svg")

![Salary Distribution](_output/figure-q2.svg)  

Insights from this visualization
 
1. Public Administration and Educational Services are the industries with lowest average paying level
2. Information sector offers the highest average salary

# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [4]:
# Your code for 3rd question here
pdf = df.select("POSTED")\
        .filter(df.POSTED.isNotNull())\
        .groupBy("POSTED")\
        .count()\
        .orderBy("POSTED")\
        .toPandas()
fig = px.line(pdf, x="POSTED", y="count",
             title="Job Posting Trends Over Time",
             color_discrete_sequence=px.colors.qualitative.Prism,
             labels={
                 "POSTED": "Posted Time",
                 "count": "Number of Job Postings",
             })
fig.update_layout(font_family="Times New Roman",
                  title_font_size=20,
                  width=1200,
                  height=600)
fig.write_image("_output/figure-q3.svg")

![Job Posting Trends Over Time](_output/figure-q3.svg)  

Insights from the data:

1. the fluctuation of job posting numbers has a weekly periodic pattern
2. most of the jobs are posted on friday

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [5]:
# Your code for 4th question here
pdf = df.select("TITLE_NAME")\
        .filter(df.TITLE_NAME.isNotNull())\
        .groupBy("TITLE_NAME")\
        .count()\
        .orderBy("count", ascending=False)\
        .limit(10)\
        .toPandas()
fig = px.bar(pdf, x="TITLE_NAME", y="count",
             title="Top 10 Job Titles",
             color_discrete_sequence=px.colors.qualitative.Prism,
             labels={
                 "TITLE_NAME": "Job Title",
                 "count": "Job Count",
             })
fig.update_layout(font_family="Times New Roman",
                  title_font_size=20,
                  width=1000,
                  height=600)
fig.write_image("_output/figure-q4.svg")

![Top 10 Job Titles by Count](_output/figure-q4.svg)  

Insights from the data:

1. Data Analyst is the most popular job
2. Most of the popular jobs have something to do with data science

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [6]:
# Your code for 5th question here
pdf = df.select("REMOTE_TYPE_NAME")\
        .filter(df.REMOTE_TYPE_NAME.isNotNull())\
        .groupBy("REMOTE_TYPE_NAME")\
        .count()\
        .toPandas()
fig = px.pie(pdf, names="REMOTE_TYPE_NAME", values="count",
             title="Remote v.s. On-Site Jobs",
             color_discrete_sequence=px.colors.qualitative.Prism,
             labels={
                 "REMOTE_TYPE_NAME": "Remote/On-Site Type",
                 "count": "Job Count",
             })
fig.update_layout(font_family="Times New Roman",
                  title_font_size=20,
                  width=800,
                  height=600)
fig.write_image("_output/figure-q5.svg")

![Remote vs On-Site Jobs](_output/figure-q5.svg)  

Insights from the data:

1. most of the job postings do not specify remote/on-site type, which may indicate on-site working
2. very little job postings explictly rule out remote working

# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [7]:
import ast

pdf = df.select("SKILLS_NAME", "NAICS2_NAME")\
        .filter(df.SKILLS.isNotNull())\
        .toPandas()

pdf["SKILLS_NAME"] = pdf["SKILLS_NAME"].apply(ast.literal_eval)
pdf = pdf.explode("SKILLS_NAME").reset_index(drop=True)

skill_industry_count = pdf.groupby(["NAICS2_NAME", "SKILLS_NAME"]).size().reset_index(name='count')
skill_industry_count = skill_industry_count\
    .sort_values(['NAICS2_NAME', 'count'], ascending=[True, False])\
    .groupby('NAICS2_NAME')\
    .head(3)\
    .reset_index(drop=True)

In [8]:
# Your code for 6th question here
fig = px.bar(skill_industry_count, x="NAICS2_NAME", y="count",
             title="Skill Demand Analysis by Industry",
             color="SKILLS_NAME",
             color_discrete_sequence=px.colors.qualitative.Prism,
             text="SKILLS_NAME",
             labels={
                 "NAICS2_NAME": "Industry",
                 "count": "Job Count",
             })
fig.update_layout(font_family="Times New Roman",
                  title_font_size=20,
                  width=1200,
                  height=1000)
fig.write_image("_output/figure-q6.svg")

![Skill Demand By Industry](_output/figure-q6.svg)   

Insights from the data:

1. Management and Communications are the most needed skill in every industry, showing the importance of soft-skills
2. Data Analysis is highly favorable across many industries

# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

**Correction:**
For Question 7, the ONET_NAME has only one possible value, Business Intelligence Analysts. This means the result bubble chart contains only one bubble. -  you can use the columns - "SALARY_FROM", "SALARY_TO", "LOT_V6_OCCUPATION_NAME", Average_Salary"


from Prof. Nakul Padalkar

In [9]:
# Your code for 7th question here
from pyspark.sql.functions import median, count, col, when

pdf = df.select("LOT_V6_OCCUPATION_NAME", "SALARY", "SALARY_FROM", "SALARY_TO")\
        .withColumn("SALARY",when(col("SALARY").isNull(), (col("SALARY_FROM")+col("SALARY_TO")) / 2.0).otherwise(col("SALARY")))\
        .select("LOT_V6_OCCUPATION_NAME", "SALARY")\
        .filter(df.SALARY.isNotNull())\
        .groupBy("LOT_V6_OCCUPATION_NAME")\
        .agg(median("SALARY").alias("MEDIAN_SALARY"), count("*").alias("JOB_COUNT"))\
        .toPandas()

pdf.head(10)

,LOT_V6_OCCUPATION_NAME,MEDIAN_SALARY,JOB_COUNT
0,Business / Management Analyst,93650.0,1640
1,Business Intelligence Analyst,125900.0,12402
2,Market Research Analyst,94500.0,65
3,Computer Systems Engineer / Architect,157600.0,3321
4,Data / Data Mining Analyst,95250.0,13286
5,Clinical Analyst / Clinical Documentation and ...,89440.0,94


In [10]:
fig = px.scatter(pdf, x="LOT_V6_OCCUPATION_NAME", y="MEDIAN_SALARY",
                 size="JOB_COUNT",
                 color="MEDIAN_SALARY",
                 title="Median Salary by Occupation Type",
                 labels={
                     "LOT_V6_OCCUPATION_NAME": "Occupation Type",
                     "MEDIAN_SALARY": "median salary",
                     "JOB_COUNT": "number of jobs",
                 })

fig.update_layout(font_family="Times New Roman",
                  title_font_size=20,
                  width=800,
                  height=600)
fig.write_image("_output/figure-q7.svg")

![Median Salary by Occupation Type](_output/figure-q7.svg)  

Insights from the data:  
 
1. Computer Systems Engineer / Architect is the Occupation Type offering the highest median salary
2. Business Intelligence Analyst jobs are of high demand and relatively high salary

# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

**Correction:**

you can use the columns - "SALARY_FROM", "SALARY_TO", "LOT_V6_OCCUPATION_NAME", Average_Salary"
For Q 8 as well these three columns will work nicely. 

from Prof. Nakul Padalkar

In [17]:
# Your code for 8th question here
from pyspark.sql.functions import approx_percentile
pdf = df.select("LOT_V6_OCCUPATION_NAME", "SALARY")\
        .filter(df.SALARY.isNotNull())\
        .groupBy("LOT_V6_OCCUPATION_NAME")\
        .agg(approx_percentile("SALARY", 0.25).alias("25%"),\
             approx_percentile("SALARY", 0.50).alias("50%"),\
             approx_percentile("SALARY", 0.75).alias("75%"))\
        .toPandas()

pdf.head(12)

,LOT_V6_OCCUPATION_NAME,25%,50%,75%
0,Business / Management Analyst,71150,93650,123000
1,Business Intelligence Analyst,100300,125900,150978
2,Market Research Analyst,89107,94500,106879
3,Computer Systems Engineer / Architect,136950,157600,187200
4,Data / Data Mining Analyst,72829,95244,123800
5,Clinical Analyst / Clinical Documentation and ...,75550,89440,105400


In [23]:
occupation_names = pdf["LOT_V6_OCCUPATION_NAME"].tolist()
len(occupation_names)

6

In [12]:
# reference: https://plotly.com/python/sankey-diagram/
import plotly.graph_objects as go

labels = pdf['SOC_2021_2_NAME'].unique().tolist() + pdf['SOC_2021_3_NAME'].unique().tolist()
label_to_index = {label: index for index, label in enumerate(labels)}

source = pdf['SOC_2021_2_NAME'].map(label_to_index)
target = pdf['SOC_2021_3_NAME'].map(label_to_index)
value = pdf['count']


fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=list(pdf['SOC_2021_2_NAME'].unique()) + list(pdf['SOC_2021_3_NAME'].unique()),
        color="blue"
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
    )
)])

fig.update_layout(title_text="Career Pathway Trends Between SOC Occupation Levels",
                  font_size=20,
                  font_family="Times New Roman",
                  width=1200,
                  height=900)

fig.write_image("_output/figure-q8.svg")

![Career Pathway Trends Between SOC Occupation Levels](_output/figure-q8.svg)

Insights from the data:

1. The data contains only one SOC_2021_2_NAME
2. The data contains only one SOC_2021_2_NAME
3. All Computer and Mathematical Occupations become Mathematical Science Occupations	